# A value history player widget

Consider a value widget `w`.
Player(`w`) wraps this widget to maintain a history of all the values that have been assigned to it.
In addition, it adds controls analogous to those of audio/video/... players to
replay that history.

In [68]:
%run HistoryPlayerWidget.py

In [69]:
w = Text()
p = Player(w)
p

Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

MainThread -1 1000000 1 True []
update thread -1 1000001 1 True []
MainThread -1 1000001 1 True []
update thread -1 1000002 1 True []


In [ ]:
with p._model._lock:
    p._model._lock.notify()

In [17]:
p._model.print()

MainThread 0 1000000 1 True ['6']


In [64]:
p.value = "6"

MainThread 10 1000000 1 True ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81']
update thread 10 1000000 1 False ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81', '6']
update thread 11 1000000 1 False ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81', '6']


In [9]:
p._model._history

['3', '5']

In [65]:
for i in range(10):
    #
    print(i)
    p.value = str(i**2)

0
MainThread 11 11 1000000 True ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81', '6']
1
update threadMainThread  11 11 1000000 False ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81', '6', '0']
11 11 1000000 False ['6', '0', '1', '4', '9', '16', '25', '36', '49', '64', '81', '6', '0']


KeyboardInterrupt: 

In [6]:
p = Player(Text())
def mysort(l):
    p.value = str(l)
    for i in range(len(l)):
        for j in range(i, 0, -1):
            if l[j-1] <= l[j]:
                break
            l[j-1], l[j] = l[j], l[j-1]
            p.value = str(l)
    return l
p

Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

In [9]:
mysort([6,5,4,3,2,1])

Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command
Running command


[1, 2, 3, 4, 5, 6]

In [18]:
p._model._clock

In [6]:
import threads

ModuleNotFoundError: No module named 'threads'

In [7]:
import lock

ModuleNotFoundError: No module named 'lock'

In [8]:
import threading

In [10]:
threading.Condition()

<Condition(<unlocked _thread.RLock object owner=0 count=0 at 0x7ff9d5e8fb40>, 0)>

In [21]:
import ipywidgets

In [22]:
hbox = ipywidgets.HBox([])

In [23]:
hbox

HBox()

In [27]:
hbox.children=[ipywidgets.IntSlider(3), ipywidgets.IntSlider(4)]

In [29]:
hbox.children

(IntSlider(value=3), IntSlider(value=4))